In [1]:
import pandas as pd
import numpy as np

# Import Data

In [2]:
# KAGGLE DATA
seeds24 = pd.read_csv('2024_tourney_seeds.csv')
mm_results = pd.read_csv('MNCAATourneyDetailedResults.csv')
mm_seeds = pd.read_csv('MNCAATourneySeeds.csv')
teams = pd.read_csv('MTeams.csv')
team_spellings = pd.read_csv('MTeamSpellings.csv', encoding='latin1')

In [3]:
# OUTSIDE DATA
cbb = pd.read_csv('cbb.csv')
cbb24 = pd.read_csv('cbb24.csv')
kenpom = pd.read_csv('KenPom Barttorvik.csv')
resumes = pd.read_csv('Resumes.csv')
upsets = pd.read_csv('Upset Count.csv')

kpi17 = pd.read_excel('KPI-Rankings_2017.xlsx')
kpi18 = pd.read_excel('KPI-Rankings_2018.xlsx')
kpi19 = pd.read_excel('KPI-Rankings_2019.xlsx')
kpi21 = pd.read_excel('KPI-Rankings_2021.xlsx')
kpi22 = pd.read_excel('KPI-Rankings_2022.xlsx')
kpi23 = pd.read_excel('KPI-Rankings_2023.xlsx')
kpi24 = pd.read_excel('KPI-Rankings_2024.xlsx')

In [4]:
# combine KPI data frames
kpi17 = kpi17[['KPI', 'Team', 'KPI #', 'SOS', 'SOS Rk']]
kpi17['Year'] = '2017'
kpi18 = kpi18[['KPI', 'Team', 'KPI #', 'SOS', 'SOS Rk']]
kpi18['Year'] = '2018'
kpi19 = kpi19[['KPI', 'Team', 'KPI #', 'SOS', 'SOS Rk']]
kpi19['Year'] = '2019'
kpi21 = kpi21[['KPI', 'Team', 'KPI #', 'SOS', 'SOS Rk']]
kpi21['Year'] = '2021'
kpi22 = kpi22[['KPI', 'Team', 'KPI #', 'SOS', 'SOS Rk']]
kpi22['Year'] = '2022'
kpi23 = kpi23[['KPI', 'Team', 'KPI #', 'SOS', 'SOS Rk']]
kpi23['Year'] = '2023'

kpi = pd.concat([kpi17, kpi18, kpi19, kpi21, kpi22, kpi23], ignore_index=True)

In [5]:
kenpom24 = kenpom[kenpom['YEAR'] == 2024]
resumes24 = resumes[resumes['YEAR'] == 2024]

kenpom = kenpom[kenpom['YEAR'] != 2024]
resumes = resumes[resumes['YEAR'] != 2024]

In [6]:
# LIMITATIONS:

# NET rankings only go back to 2021 tournament: https://stats.ncaa.org/selection_rankings/nitty_gritties
# CBB rankings only go back to 2013 tournament
# KPI rankings only go back to 2017 tournament: https://faktorsports.com/
# 538 shut down for 2024, so can't use power ratings

# Merge Tournament Results Data

In [7]:
# isolate relevant columns
mm_results = mm_results[['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore']]

# merge team names to match ID's
mm_results = mm_results.merge(teams, left_on='WTeamID', right_on='TeamID', how='left', suffixes=('', '_W'))
mm_results = mm_results.merge(teams, left_on='LTeamID', right_on='TeamID', how='left', suffixes=('', '_L'))

# isolate and rename relevant columns 
mm_results.drop(['TeamID', 'TeamID_L'], axis=1, inplace=True)
mm_results.rename(columns={'TeamName': 'WTeamName', 'TeamName_L': 'LTeamName'}, inplace=True)
mm_results.drop(['FirstD1Season', 'LastD1Season', 'FirstD1Season_L', 'LastD1Season_L'], axis=1, inplace=True)

# filter for relevant matches
year_mask = (mm_results['Season'] >= 2017)
round_mask = (mm_results['DayNum'] >= 136)
mm_results = mm_results[year_mask & round_mask]

In [8]:
# create column for round
def get_round(day_num):
    if day_num in [134, 135]:
        return 'Play-in games'
    elif day_num in [136, 137]:
        return 'Round 1'
    elif day_num in [138, 139]:
        return 'Round 2'
    elif day_num in [143, 144]:
        return 'Round 3 (Sweet Sixteen)'
    elif day_num in [145, 146]:
        return 'Round 4 (Elite Eight)'
    elif day_num == 152:
        return 'Round 5 (Final Four)'
    elif day_num == 154:
        return 'Round 6 (National Final)'
    else:
        return 'Unknown'

mm_results['Round'] = mm_results['DayNum'].apply(get_round)

In [9]:
# remove region from seeds data
mm_seeds['Seed'] = mm_seeds['Seed'].apply(lambda x: int(x[1:3]))

In [10]:
# merge tournament seeds onto teams
mm_results = mm_results.merge(mm_seeds, left_on=['Season','WTeamID'], right_on=['Season','TeamID'], how='left', suffixes=('', '_W'))
mm_results = mm_results.merge(mm_seeds, left_on=['Season','LTeamID'], right_on=['Season','TeamID'], how='left', suffixes=('', '_L'))

# isolate and rename relevant columns 
mm_results.drop(['TeamID', 'TeamID_L'], axis=1, inplace=True)
mm_results.rename(columns={'Seed': 'WSeed', 'Seed_L': 'LSeed'}, inplace=True)

In [11]:
def rename_teams(row):
    sorted_names = sorted([row['WTeamName'], row['LTeamName']])
    return pd.Series({'ATeamName': sorted_names[0], 'BTeamName': sorted_names[1]})

new_columns = mm_results.apply(rename_teams, axis=1)
mm_results = pd.concat([mm_results, new_columns], axis=1)

In [12]:
def match_team_info(row):
    if row['WTeamName'] == row['ATeamName']:
        return pd.Series({'ATeamID': row['WTeamID'], 'BTeamID': row['LTeamID'],
                          'AScore': row['WScore'], 'BScore': row['LScore'],
                          'ASeed': row['WSeed'], 'BSeed': row['LSeed']})
    elif row['WTeamName'] == row['BTeamName']:
        return pd.Series({'BTeamID': row['WTeamID'], 'ATeamID': row['LTeamID'],
                          'BScore': row['WScore'], 'AScore': row['LScore'],
                          'BSeed': row['WSeed'], 'ASeed': row['LSeed']})

new_columns = mm_results.apply(match_team_info, axis=1)
mm_results = pd.concat([mm_results, new_columns], axis=1)
mm_results.drop(['WTeamName','LTeamName','WTeamID','LTeamID','WScore','LScore','WSeed','LSeed'], axis=1, inplace=True)

# Merge Team Stats Data

In [13]:
resumes = resumes[resumes['YEAR'] >= 2017]
cbb = cbb[cbb['YEAR'] >= 2017]
kenpom = kenpom[(kenpom['YEAR'] >= 2017) & (kenpom['YEAR'] <= 2023)]

In [14]:
resumes['YEAR'] = resumes['YEAR'].astype(str)
cbb['YEAR'] = cbb['YEAR'].astype(str)
kenpom['YEAR'] = kenpom['YEAR'].astype(str)
team_spellings['TeamID'] = team_spellings['TeamID'].astype(str)

In [15]:
corrections = {'illinois chicago':'1227',
               'bethune cookman':'1126',
               'maryland eastern shore':'1271',
               'st. francis ny':'1383',
               'st. francis pa':'1384',
               'tennessee martin':'1404',
               'louisiana monroe':'1419',
               'ut rio grande valley':'1410',
               'dixie st.':'1469',
               'tarleton st.':'1470',
               'st. thomas':'1472',
               'texas a&m corpus chris':'1394',
               'tamu-corpus christi':'1394',        
               'louisiana lafayette':'1418',
               'southeast missouri st.':'1369',
               'cal st. bakersfield':'1167',
               'cal state-bakersfield':'1167',
               'arkansas pine bluff':'1115',
               'mississippi valley st.':'1290',
               "st. mary's (ca)":'1388',
               'unc-wilmington':'1423',
               'unc-greensboro':'1422',
               'lasalle':'1247',
               'tex. a&m-commerce':'1477',
               'texas a&m commerce':'1477',
               'lindenwood (mo)':'1473',
               'cal state-northridge':'1169',
               'houston christian':'1223',
               'purdue-fort wayne':'1236',
               'loyola (chicago)':'1260',
               'md-eastern shore':'1271',
               'uc-san diego':'1471',
               'st. thomas (mn)':'1472',
               'queens (nc)':'1474',
               'queens':'1474',
               'southern ind.':'1475'}

for team,ID in corrections.items():
    new_row = pd.DataFrame([[team, ID]], columns=['TeamNameSpelling', 'TeamID'])
    team_spellings = pd.concat([team_spellings, new_row], ignore_index=True)

In [16]:
# create team data frame
team_data = kpi[['Team', 'Year', 'KPI #', 'SOS']]

# get team name into a format compatible with team_spellings data frame
team_data['Team'] = team_data['Team'].str.replace(r'\bSt\.\b', 'State').apply(lambda x: x.lower())

# add TeamID
team_data = team_data.merge(team_spellings, left_on=['Team'], right_on=['TeamNameSpelling'], how='left')
team_data.drop(['TeamNameSpelling'], axis=1, inplace=True)
#team_data[team_data['TeamID'].isna()] # make sure all teams received a teamID

C:\Users\ethan\AppData\Local\Temp\ipykernel_16620\3275743952.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  team_data['Team'] = team_data['Team'].str.replace(r'\bSt\.\b', 'State').apply(lambda x: x.lower())
C:\Users\ethan\AppData\Local\Temp\ipykernel_16620\3275743952.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_data['Team'] = team_data['Team'].str.replace(r'\bSt\.\b', 'State').apply(lambda x: x.lower())


In [17]:
# isolate relevant columns for remaining relevant data frames
resumes = resumes[['YEAR','TEAM','NET RPI','RESUME','WAB RANK','ELO','B POWER', 'Q1 W', 'Q1 PLUS Q2 W', 'Q3 Q4 L', 'R SCORE']]

# get team name into a format compatible with team_spellings data frame
resumes['TEAM'] = resumes['TEAM'].str.replace(r'\bSt\.\b', 'State').apply(lambda x: x.lower())

# add TeamID
resumes = resumes.merge(team_spellings, left_on=['TEAM'], right_on=['TeamNameSpelling'], how='left')
resumes.drop(['TeamNameSpelling'], axis=1, inplace=True)
resumes[resumes['TeamID'].isna()] # make sure all teams received a teamID

C:\Users\ethan\AppData\Local\Temp\ipykernel_16620\2234263552.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  resumes['TEAM'] = resumes['TEAM'].str.replace(r'\bSt\.\b', 'State').apply(lambda x: x.lower())


,YEAR,TEAM,NET RPI,RESUME,WAB RANK,ELO,B POWER,Q1 W,Q1 PLUS Q2 W,Q3 Q4 L,R SCORE,TeamID


In [18]:
# add resume data
team_data = team_data.merge(resumes, left_on=['TeamID','Year'], right_on=['TeamID','YEAR'], how='outer')
team_data.drop(['TEAM', 'YEAR'], axis=1, inplace=True)

In [19]:
# isolate relevant columns for remaining relevant data frames
cbb = cbb[['YEAR', 'TEAM', 'ADJOE','ADJDE','BARTHAG','EFG_O','EFG_D','TOR','TORD', 'ORB', 'DRB', 'FTR', 'FTRD', \
           '2P_O', '2P_D', '3P_O', '3P_D', 'ADJ_T', 'WAB']]

# get team name into a format compatible with team_spellings data frame
cbb['TEAM'] = cbb['TEAM'].str.replace(r'\bSt\.\b', 'State').apply(lambda x: x.lower())

# add TeamID
cbb = cbb.merge(team_spellings, left_on=['TEAM'], right_on=['TeamNameSpelling'], how='left')
cbb.drop(['TeamNameSpelling'], axis=1, inplace=True)
cbb[cbb['TeamID'].isna()] # make sure all teams received a teamID

C:\Users\ethan\AppData\Local\Temp\ipykernel_16620\1733176395.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  cbb['TEAM'] = cbb['TEAM'].str.replace(r'\bSt\.\b', 'State').apply(lambda x: x.lower())


,YEAR,TEAM,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,ORB,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,TeamID


In [20]:
# add cbb data
team_data = team_data.merge(cbb, left_on=['TeamID','Year'], right_on=['TeamID','YEAR'], how='outer')
team_data.drop(['TEAM', 'YEAR'], axis=1, inplace=True)

In [21]:
# isolate relevant columns for remaining relevant data frames
kenpom = kenpom[['YEAR','QUAD ID','TEAM','K TEMPO','KADJ T','K OFF','KADJ O','K DEF','KADJ D','KADJ EM','BADJ EM', \
                'BADJ O','BADJ D','EFG%','TOV%','TOV%D','OREB%','DREB%','OP OREB%','OP DREB%','BLK%','BLKED%', \
                'AST%','OP AST%','BADJ T','AVG HGT','EFF HGT','EXP','TALENT','FT%', 'PPPO', 'PPPD','ELITE SOS']]

# get team name into a format compatible with team_spellings data frame
kenpom['TEAM'] = kenpom['TEAM'].str.replace(r'\bSt\.\b', 'State').apply(lambda x: x.lower())

# add TeamID
kenpom = kenpom.merge(team_spellings, left_on=['TEAM'], right_on=['TeamNameSpelling'], how='left')
kenpom.drop(['TeamNameSpelling'], axis=1, inplace=True)
kenpom[kenpom['TeamID'].isna()] # make sure all teams received a teamID

C:\Users\ethan\AppData\Local\Temp\ipykernel_16620\3773206749.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  kenpom['TEAM'] = kenpom['TEAM'].str.replace(r'\bSt\.\b', 'State').apply(lambda x: x.lower())


,YEAR,QUAD ID,TEAM,K TEMPO,KADJ T,K OFF,KADJ O,K DEF,KADJ D,KADJ EM,...,BADJ T,AVG HGT,EFF HGT,EXP,TALENT,FT%,PPPO,PPPD,ELITE SOS,TeamID


In [22]:
# add cbb data
team_data = team_data.merge(kenpom, left_on=['TeamID','Year'], right_on=['TeamID','YEAR'], how='outer')
team_data.drop(['TEAM', 'YEAR'], axis=1, inplace=True)

# Repeat Process for 2024 Data

In [23]:
# create team data frame
team_data24 = kpi24[['Team', 'KPI #', 'SOS']]

# get team name into a format compatible with team_spellings data frame
team_data24['Team'] = team_data24['Team'].str.replace(r'\bSt\.\b', 'State').apply(lambda x: x.lower())

# add TeamID
team_data24 = team_data24.merge(team_spellings, left_on=['Team'], right_on=['TeamNameSpelling'], how='left')
team_data24.drop(['TeamNameSpelling'], axis=1, inplace=True)
team_data24[team_data24['TeamID'].isna()] # make sure all teams received a teamID

C:\Users\ethan\AppData\Local\Temp\ipykernel_16620\2035469998.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  team_data24['Team'] = team_data24['Team'].str.replace(r'\bSt\.\b', 'State').apply(lambda x: x.lower())
C:\Users\ethan\AppData\Local\Temp\ipykernel_16620\2035469998.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_data24['Team'] = team_data24['Team'].str.replace(r'\bSt\.\b', 'State').apply(lambda x: x.lower())


,Team,KPI #,SOS,TeamID
359,non d-i,-0.37,0.009,NaN


In [24]:
# isolate relevant columns for remaining relevant data frames
resumes24 = resumes24[['TEAM','NET RPI','RESUME','WAB RANK','ELO','B POWER', 'Q1 W', 'Q1 PLUS Q2 W', 'Q3 Q4 L', 'R SCORE']]

# get team name into a format compatible with team_spellings data frame
resumes24['TEAM'] = resumes24['TEAM'].str.replace(r'\bSt\.\b', 'State').apply(lambda x: x.lower())

# add TeamID
resumes24 = resumes24.merge(team_spellings, left_on=['TEAM'], right_on=['TeamNameSpelling'], how='left')
resumes24.drop(['TeamNameSpelling'], axis=1, inplace=True)
resumes24[resumes24['TeamID'].isna()] # make sure all teams received a teamID

C:\Users\ethan\AppData\Local\Temp\ipykernel_16620\1594971604.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  resumes24['TEAM'] = resumes24['TEAM'].str.replace(r'\bSt\.\b', 'State').apply(lambda x: x.lower())


,TEAM,NET RPI,RESUME,WAB RANK,ELO,B POWER,Q1 W,Q1 PLUS Q2 W,Q3 Q4 L,R SCORE,TeamID


In [25]:
# add resume data
team_data24 = team_data24.merge(resumes24, left_on=['TeamID'], right_on=['TeamID'], how='outer')
team_data24.drop(['TEAM'], axis=1, inplace=True)

In [26]:
# isolate relevant columns for remaining relevant data frames
cbb24.rename(columns={'EFG%': 'EFG_O', 'EFGD%': 'EFG_D'}, inplace=True)
cbb24 = cbb24[['TEAM', 'ADJOE','ADJDE','BARTHAG','EFG_O','EFG_D','TOR','TORD', 'ORB', 'DRB', 'FTR', 'FTRD', \
               '2P_O', '2P_D', '3P_O', '3P_D', 'ADJ_T', 'WAB']]

# get team name into a format compatible with team_spellings data frame
cbb24['TEAM'] = cbb24['TEAM'].str.replace(r'\bSt\.\b', 'State').apply(lambda x: x.lower())

# add TeamID
cbb24 = cbb24.merge(team_spellings, left_on=['TEAM'], right_on=['TeamNameSpelling'], how='left')
cbb24.drop(['TeamNameSpelling'], axis=1, inplace=True)
cbb24[cbb24['TeamID'].isna()] # make sure all teams received a teamID

C:\Users\ethan\AppData\Local\Temp\ipykernel_16620\43191826.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  cbb24['TEAM'] = cbb24['TEAM'].str.replace(r'\bSt\.\b', 'State').apply(lambda x: x.lower())


,TEAM,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,ORB,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,TeamID


In [27]:
# add cbb data
team_data24 = team_data24.merge(cbb24, left_on=['TeamID'], right_on=['TeamID'], how='outer')
team_data24.drop(['TEAM'], axis=1, inplace=True)

In [28]:
# isolate relevant columns for remaining relevant data frames
kenpom24 = kenpom24[['QUAD ID','TEAM','K TEMPO','KADJ T','K OFF','KADJ O','K DEF','KADJ D','KADJ EM','BADJ EM', \
                     'BADJ O','BADJ D','EFG%','TOV%','TOV%D','OREB%','DREB%','OP OREB%','OP DREB%','BLK%','BLKED%', \
                     'AST%','OP AST%','BADJ T','AVG HGT','EFF HGT','EXP','TALENT','FT%', 'PPPO', 'PPPD','ELITE SOS']]

# get team name into a format compatible with team_spellings data frame
kenpom24['TEAM'] = kenpom24['TEAM'].str.replace(r'\bSt\.\b', 'State').apply(lambda x: x.lower())

# add TeamID
kenpom24 = kenpom24.merge(team_spellings, left_on=['TEAM'], right_on=['TeamNameSpelling'], how='left')
kenpom24.drop(['TeamNameSpelling'], axis=1, inplace=True)
kenpom24[kenpom24['TeamID'].isna()] # make sure all teams received a teamID

C:\Users\ethan\AppData\Local\Temp\ipykernel_16620\4168628599.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  kenpom24['TEAM'] = kenpom24['TEAM'].str.replace(r'\bSt\.\b', 'State').apply(lambda x: x.lower())


,QUAD ID,TEAM,K TEMPO,KADJ T,K OFF,KADJ O,K DEF,KADJ D,KADJ EM,BADJ EM,...,BADJ T,AVG HGT,EFF HGT,EXP,TALENT,FT%,PPPO,PPPD,ELITE SOS,TeamID


In [29]:
# add cbb data
team_data24 = team_data24.merge(kenpom24, left_on=['TeamID'], right_on=['TeamID'], how='outer')
team_data24.drop(['TEAM'], axis=1, inplace=True)

# Add Team Stats Back to Results

In [30]:
mm_results['Season'] = mm_results['Season'].astype(str)
mm_results['ATeamID'] = mm_results['ATeamID'].astype(str)
mm_results['BTeamID'] = mm_results['BTeamID'].astype(str)

In [31]:
def winner(row):
    if row['AScore'] > row['BScore']:
        return True
    elif row['BScore'] > row['AScore']:
        return False

mm_results['AWon'] = mm_results.apply(winner, axis=1)

In [32]:
mm_results = mm_results.merge(team_data, left_on=['ATeamID','Season'], right_on=['TeamID','Year'], how='left')
mm_results = mm_results.merge(team_data, left_on=['BTeamID','Season'], right_on=['TeamID','Year'], how='left', suffixes=('_A', '_B'))

In [33]:
# make sure there are zero NA values in the final data set for training the neural network
mm_results.isna().sum().sum()

0

In [34]:
training_df = mm_results.drop(['DayNum','Round','AScore','BScore','ATeamID','BTeamID','Team_A','Team_B', \
                               'Year_A','Year_B','TeamID_A','TeamID_B'], axis=1)

# make everything except for 'Winner' a float
for col in training_df.columns:
    if col not in ['Season','ATeamName','BTeamName','AWon']:
        training_df[col] = training_df[col].astype('float64')

# Build Neural Network Model

In [35]:
import tensorflow as tf
from sklearn.model_selection import train_test_split, cross_val_score
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from scipy.optimize import minimize
from sklearn.preprocessing import MinMaxScaler

In [36]:
def upset_weights(row):
    if ((row['ASeed'] >= row['BSeed'] + 2) & (row['AWon'] == True) or \
        (row['BSeed'] >= row['ASeed'] + 2) & (row['AWon'] == False)):
        return 293/88
    else:
        return 1

training_df['Upset'] = training_df.apply(upset_weights, axis=1)

In [37]:
scaled_df = pd.DataFrame(MinMaxScaler(). \
                         fit_transform(training_df.drop(columns=['Season','ATeamName','BTeamName','AWon','Upset'])), \
                         columns=training_df.drop(columns=['Season','ATeamName','BTeamName','AWon','Upset']).columns)

In [38]:
scaled_df['AWon'] = training_df['AWon']
scaled_df['Upset'] = training_df['Upset']

In [39]:
# set appropriate x and y values
x = scaled_df.drop(columns=['AWon','Upset'])
y = scaled_df.AWon

Feature selection!

In [40]:
# use random forest technique for feature importance
rf = RandomForestRegressor()
rf.fit(x,y)
rf_imps = rf.feature_importances_

In [41]:
# use gradient boosting technique for feature importance
xg_reg = xgb.XGBRegressor()
xg_reg.fit(x,y)
xg_imps = xg_reg.feature_importances_

In [42]:
# combine these techniques into a dataframe
importance = pd.DataFrame({'Feature':x.columns, 'Random Forest':rf_imps, 'Gradient Boosting':xg_imps})

# average feature importance for two teams
importance['Feature'] = importance['Feature'].apply(lambda x: x[:-2])
importance.loc['ASeed', 'Feature'] = 'Seed_A'
importance.loc['BSeed', 'Feature'] = 'Seed_B'
rf_pivot = pd.pivot_table(importance, index='Feature', columns=None, values='Random Forest', aggfunc='mean')
xg_pivot = pd.pivot_table(importance, index='Feature', columns=None, values='Gradient Boosting', aggfunc='mean')
importance = rf_pivot.merge(xg_pivot, on = 'Feature')

# filter for the most important features
rf_mask = (importance['Random Forest'] >= importance['Random Forest'].median())
gb_mask = (importance['Gradient Boosting'] >= importance['Gradient Boosting'].median())
importance = importance[rf_mask & gb_mask]

In [43]:
imp_columns = [column for column in training_df.columns if any(name in column for name in importance.index)]
x = scaled_df[imp_columns]
y = scaled_df.AWon
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.33,random_state=3)

In [44]:
# build neural network model
NNmodel = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32,activation='relu',input_shape=(len(x.columns),)),
    tf.keras.layers.Dense(16,activation='relu'),
    tf.keras.layers.Dense(8,activation='relu'),
    tf.keras.layers.Dense(1,activation=tf.nn.sigmoid)])

NNmodel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss='binary_crossentropy',
              metrics=['accuracy'],
              weighted_metrics=[]
               )

In [45]:
merged_df = xtrain.merge(scaled_df, left_index=True, right_index=True)
xtrain['Upset'] = merged_df['Upset']

In [46]:
# fit the model
NNmodel.fit(xtrain.drop(columns='Upset'), \
            ytrain, epochs=20, batch_size=10, validation_split=0.33, \
            sample_weight=xtrain['Upset']
           )

Epoch 1/20


17/17 [==============================] - 2s 20ms/step - loss: 1.1038 - accuracy: 0.5235 - val_loss: 1.0384 - val_accuracy: 0.5294
Epoch 2/20
17/17 [==============================] - 0s 6ms/step - loss: 1.0539 - accuracy: 0.5235 - val_loss: 1.0719 - val_accuracy: 0.4824
Epoch 3/20
17/17 [==============================] - 0s 6ms/step - loss: 1.0380 - accuracy: 0.5176 - val_loss: 1.0573 - val_accuracy: 0.4824
Epoch 4/20
17/17 [==============================] - 0s 7ms/step - loss: 1.0296 - accuracy: 0.5176 - val_loss: 1.0517 - val_accuracy: 0.4824
Epoch 5/20
17/17 [==============================] - 0s 10ms/step - loss: 1.0375 - accuracy: 0.5176 - val_loss: 1.0683 - val_accuracy: 0.4824
Epoch 6/20
17/17 [==============================] - 0s 7ms/step - loss: 1.0271 - accuracy: 0.6059 - val_loss: 1.0472 - val_accuracy: 0.7059
Epoch 7/20
17/17 [==============================] - 0s 7ms/step - loss: 1.0234 - accuracy: 0.7000 - val_loss: 1.0951 - val_accuracy: 0.5647
Epoch 8/20
17/17

In [47]:
# evaluate model testing accuracy
print('This model predicts '+str(round(NNmodel.evaluate(xtest,ytest)[1]*100,4)) +'% of the test data correctly')

4/4 [==============================] - 0s 4ms/step - loss: 0.6652 - accuracy: 0.5556
This model predicts 55.5556% of the test data correctly


In [48]:
# display the games where team A is most and least likely to win
training_df['AProb'] = NNmodel.predict(x)
training_df[(training_df['ASeed'].isin([5.0,12.0])) & (training_df['BSeed'].isin([5.0,12.0]))].sort_values(by='AProb', ascending=False)
training_df.sort_values(by='AProb', ascending=False)

12/12 [==============================] - 0s 3ms/step


,Season,ATeamName,BTeamName,ASeed,BSeed,AWon,KPI #_A,SOS_A,NET RPI_A,RESUME_A,...,AVG HGT_B,EFF HGT_B,EXP_B,TALENT_B,FT%_B,PPPO_B,PPPD_B,ELITE SOS_B,Upset,AProb
318,2023,Alabama,TAM C. Christi,1.0,16.0,True,0.489,0.198,2.0,3.0,...,75.074,78.271,2.721,3.833,80.0,1.098,1.023,9.546,1.000000,0.666133
352,2023,Auburn,Houston,9.0,1.0,False,0.259,0.185,32.0,49.0,...,76.743,79.511,1.611,68.424,71.9,1.158,0.872,23.376,1.000000,0.664826
227,2021,Illinois,Loyola-Chicago,1.0,8.0,False,0.329,0.148,3.0,1.0,...,76.146,79.589,2.231,11.054,73.3,1.102,0.873,16.033,3.329545,0.664141
167,2019,Houston,Ohio St,3.0,11.0,True,0.401,0.090,4.0,16.0,...,76.691,79.478,1.460,56.419,73.4,1.037,0.986,33.733,1.000000,0.663094
380,2023,Connecticut,San Diego St,4.0,5.0,True,0.287,0.100,8.0,19.0,...,76.704,80.193,2.718,37.236,73.6,1.064,0.952,25.602,1.000000,0.661831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,2017,Mt St Mary's,Villanova,16.0,1.0,False,-0.019,-0.061,150.0,218.0,...,77.636,79.413,1.845,75.837,79.4,1.180,0.954,30.578,1.000000,0.109585
147,2019,Iona,North Carolina,16.0,1.0,False,-0.087,-0.120,202.0,273.0,...,78.502,80.814,1.759,76.654,74.2,1.126,0.955,36.978,1.000000,0.094353
130,2019,F Dickinson,Gonzaga,16.0,1.0,False,-0.065,-0.152,203.0,263.0,...,78.212,80.475,1.839,55.473,76.7,1.241,0.909,21.704,1.000000,0.091899
94,2018,TX Southern,Xavier,16.0,1.0,False,-0.102,-0.066,222.0,245.0,...,78.483,80.776,1.844,76.280,79.0,1.155,1.021,31.853,1.000000,0.073880


In [54]:
imp_columns

['SOS_A',
 'NET RPI_A',
 'RESUME_A',
 'WAB RANK_A',
 'B POWER_A',
 'R SCORE_A',
 'ADJOE_A',
 'ADJDE_A',
 'BARTHAG_A',
 'FTRD_A',
 '3P_D_A',
 'KADJ O_A',
 'BADJ EM_A',
 'BADJ O_A',
 'AVG HGT_A',
 'EFF HGT_A',
 'FT%_A',
 'ELITE SOS_A',
 'SOS_B',
 'NET RPI_B',
 'RESUME_B',
 'WAB RANK_B',
 'B POWER_B',
 'R SCORE_B',
 'ADJOE_B',
 'ADJDE_B',
 'BARTHAG_B',
 'FTRD_B',
 '3P_D_B',
 'KADJ O_B',
 'BADJ EM_B',
 'BADJ O_B',
 'AVG HGT_B',
 'EFF HGT_B',
 'FT%_B',
 'ELITE SOS_B']

In [55]:
# print model variables that most strongly correlate with probability of winning
corrs = pd.DataFrame({'Feature':training_df.drop(columns=['Season','ATeamName','BTeamName']).columns,
                      'AProb':training_df.corr()['AProb']})

corrs.drop(['AWon','AProb'], inplace=True)
corrs.loc['ASeed', 'Feature'] = 'Seed_A'
corrs.loc['BSeed', 'Feature'] = 'Seed_B'

corrs['Feature'] = corrs['Feature'].apply(lambda x: x[:-2])
pivot = pd.pivot_table(corrs, index='Feature', columns=None, values='AProb', aggfunc=lambda x: abs(x).mean())

def imp(feature):
    if feature + '_A' in imp_columns:
        return True
    else:
        return False
pivot['Important'] = pivot.index.map(imp)
pivot.sort_values(by='AProb', ascending=False)[:10]

,AProb,Important
Feature,,
WAB,0.541700,False
BADJ EM,0.532291,True
KADJ EM,0.525229,False
KPI #,0.521994,False
Seed,0.516072,False
BADJ O,0.511579,True
KADJ O,0.509522,True
ADJOE,0.504469,True
BARTHAG,0.496777,True


# Apply Model to 2024 Data

In [ ]:
matchups = pd.read_excel('MM_matchups24.xlsx')

def rename_teams(row):
    sorted_names = sorted([row['HTeamName'], row['LTeamName']])
    return pd.Series({'ATeamName': sorted_names[0], 'BTeamName': sorted_names[1]})

new_columns = matchups.apply(rename_teams, axis=1)
matchups = pd.concat([matchups, new_columns], axis=1)

def match_team_info(row):
    if row['HTeamName'] == row['ATeamName']:
        return pd.Series({'ASeed': row['HSeed'], 'BSeed': row['LSeed']})
    elif row['HTeamName'] == row['BTeamName']:
        return pd.Series({'BSeed': row['HSeed'], 'ASeed': row['LSeed']})

new_columns = matchups.apply(match_team_info, axis=1)
matchups = pd.concat([matchups, new_columns], axis=1)
matchups.drop(['HTeamName','LTeamName','HSeed','LSeed'], axis=1, inplace=True)

In [ ]:
matchups['ATeamName'] = matchups['ATeamName'].str.replace(r'\bSt\.\b', 'State').apply(lambda x: x.lower())
matchups['BTeamName'] = matchups['BTeamName'].str.replace(r'\bSt\.\b', 'State').apply(lambda x: str(x).lower())

# add TeamID
matchups = matchups.merge(team_spellings, left_on=['ATeamName'], right_on=['TeamNameSpelling'], how='left')
matchups.drop(['TeamNameSpelling'], axis=1, inplace=True)

matchups = matchups.merge(team_spellings, left_on=['BTeamName'], right_on=['TeamNameSpelling'], how='left')
matchups.drop(['TeamNameSpelling'], axis=1, inplace=True)

matchups.rename(columns={'TeamID_x': 'ATeamID', 'TeamID_y': 'BTeamID'}, inplace=True)

In [ ]:
matchups = matchups.merge(team_data24, left_on=['ATeamID'], right_on=['TeamID'], how='left')
matchups = matchups.merge(team_data24, left_on=['BTeamID'], right_on=['TeamID'], how='left', suffixes=('_A', '_B'))

In [ ]:
testing_df = matchups.drop(['Region','Round','ATeamID','BTeamID','Team_A','Team_B','TeamID_A','TeamID_B'], axis=1)

# make everything except for 'Winner' a float
for col in testing_df.columns:
    if col not in ['ATeamName','BTeamName']:
        testing_df[col] = testing_df[col].astype('float64')

In [ ]:
scaled_test_df = pd.DataFrame(MinMaxScaler(). \
                              fit_transform(testing_df.drop(columns=['ATeamName','BTeamName'])), \
                              columns=testing_df.drop(columns=['ATeamName','BTeamName']).columns)

In [ ]:
x24 = scaled_test_df[imp_columns]
testing_df['AProb'] = NNmodel.predict(x24)

In [ ]:
testing_df = testing_df[['ATeamName','BTeamName','ASeed','BSeed','AProb']]

def order_seeds(row):
    if row['ASeed'] > row['BSeed']:
        return pd.Series({'HTeamName': row['BTeamName'], 'LTeamName': row['ATeamName'],
                          'HSeed': row['BSeed'], 'LSeed': row['ASeed'],
                          'HProb': 1 - row['AProb'], 'LProb': row['AProb']})
    else:
        return pd.Series({'HTeamName': row['ATeamName'], 'LTeamName': row['BTeamName'],
                          'HSeed': row['ASeed'], 'LSeed': row['BSeed'],
                          'HProb': row['AProb'], 'LProb': 1 - row['AProb']})

new_columns = testing_df.apply(order_seeds, axis=1)
testing_df = pd.concat([testing_df, new_columns], axis=1)
testing_df.drop(columns=['ATeamName','BTeamName','ASeed','BSeed','AProb'], inplace=True)

matchup_merger = matchups[['Round','ATeamName','BTeamName']]
new_columns = testing_df.apply(rename_teams, axis=1)
testing_df = pd.concat([testing_df, new_columns], axis=1)

testing_df = testing_df.merge(matchup_merger, on=['ATeamName','BTeamName'])
testing_df.drop(columns=['ATeamName','BTeamName'], inplace=True)
testing_df.sort_values(by='LProb', ascending=False)

In [ ]:
prob24_results = {}
for index, row in testing_df.iterrows():
    key = row['HTeamName']
    value = row['HProb']
    if key in prob24_results:
        prob24_results[key].append(value)
    else:
        prob24_results[key] = [value]
prob24_results

In [ ]:
# an upset is defined as a team at least two seeds worse winning (i.e. 10 vs 7 but not 9 vs 8)
print('First round upsets:', round(upsets['FIRST ROUND'].mean(),1))
print('Second round upsets:', round(upsets['SECOND ROUND'].mean(),1))
print('Sweet 16 upsets:', round(upsets['SWEET 16'].mean(),1))
print('Elite 8 upsets:', round(upsets['ELITE 8'].mean(),1))

In [ ]:
# compare to ESPN analytics

# Michigan State 57.7%
# BYU 81.7%
# Creighton 90.6%
# Arizona 97.8%
# North Carolina 98.8%
# Illinois 89.0%
# South Carolina 55.6%
# Dayton 57.3%
# Texas 71.9%
# Kentucky 90.4%
# Iowa State 96.7%
# Gonzaga 76.6%
# Tennessee 98.1%
# Texas Tech 61.3%
# Washington State 51.2%
# Kansas 83.7%
# Florida Atlantic 54.8%
# Baylor 90.6%
# San Diego State 85.0%
# Marquette 94.1%
# Clemson 57.8%
# UConn 98.7%
# Auburn 90.5%
# Florida 63.2%
# Texas A&M 52.6%
# Duke 88.3%
# Purdue 99.5%
# Alabama 90.4%
# Houston 98.6%
# Wisconsin 68.6%
# TCU 72.5%
# Saint Mary's 78.9%

# Experiment in Using Gurobi to Optimize Neural Network - Not Deployed

In [ ]:
def optimal_model(params):
    layers = int(params[0])
    nodes = int(params[1])
    dropout = params[2]
    #epochs = int(params[3])
    #batch_size = int(params[4])
    #val_split = params[5]
    
    accuracies = []
    for i in range(3):
        scaled_df = pd.DataFrame(MinMaxScaler(). \
                         fit_transform(training_df.drop(columns=['Season','ATeamName','BTeamName','AWon'])), \
                         columns=training_df.drop(columns=['Season','ATeamName','BTeamName','AWon']).columns)
        
        scaled_df['AWon'] = training_df['AWon']
        
        x = scaled_df.drop(columns='AWon')
        y = scaled_df.AWon
        xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.33,random_state=3)
    
        rf = RandomForestRegressor()
        rf.fit(x,y)
        rf_imps = rf.feature_importances_

        xg_reg = xgb.XGBRegressor()
        xg_reg.fit(x,y)
        xg_imps = xg_reg.feature_importances_

        importance = pd.DataFrame({'Feature':x.columns, 'Random Forest':rf_imps, 'Gradient Boosting':xg_imps})
        importance['Feature'] = importance['Feature'].apply(lambda x: x[:-2])
        rf_pivot = pd.pivot_table(importance, index='Feature', columns=None, values='Random Forest', aggfunc='mean')
        xg_pivot = pd.pivot_table(importance, index='Feature', columns=None, values='Gradient Boosting', aggfunc='mean')
        importance = rf_pivot.merge(xg_pivot, on = 'Feature')
        importance = importance[(importance['Random Forest'] >= 0.007) & (importance['Gradient Boosting'] >= 0.007)]

        imp_columns = [column for column in training_df.columns if any(name in column for name in importance.index)]
        x = scaled_df[imp_columns]
        y = scaled_df.AWon
        xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.33,random_state=3)
        
        sequential = []
        sequential.append(tf.keras.layers.Dense(nodes, activation='relu', input_shape=(len(x.columns),)))
        for i in range(layers):
            sequential.append(tf.keras.layers.Dense(nodes/(2**(i+1)), activation='relu'))
            sequential.append(tf.keras.layers.Dropout(dropout))
        sequential.append(tf.keras.layers.Dense(1,activation=tf.nn.sigmoid))
        
        NNmodel = tf.keras.models.Sequential(sequential)

        NNmodel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])
    
        NNmodel.fit(xtrain,ytrain,epochs=20,batch_size=10,validation_split=0.33, verbose=0)
    
        accuracy = NNmodel.evaluate(xtest,ytest)[1]*100
        accuracies.append(accuracy)
    
    mean_accuracy = round(sum(accuracies) / len(accuracies),4)
    print(f'New Iteration! Layers: {layers} Nodes: {nodes} Dropout: {dropout} \nMean Accuracy: {mean_accuracy} %')
    return -mean_accuracy

In [ ]:
optNNmodel = minimize(optimal_model, [3,64,0.1], bounds=[(0,20),(0,1000),(0,1)], options={'maxiter': 10}, tol=1e-8)

In [ ]:
print(-optNNmodel.fun)
print(optNNmodel.x)